In [123]:
#import library

import pandas as pd
import pyodbc
import sklearn
from sklearn.preprocessing import StandardScaler


In [124]:
#connect to sql server and get data
server= 'LAPTOP-RAE61A0J\SQLEXPRESS'
db1 = 'DATABASERFM'
uname = 'lvtrung289'
pword = 'lvtrung289'
conn = pyodbc.connect(driver='{SQL Server Native Client 11.0}', 
                      host=server, database=db1, trusted_connection='yes',
                      user=uname, password=pword)
cursor = conn.cursor()

data = pd.read_sql_query('select * from rfm_group', conn)

conn.close()

In [125]:
data.head()

,customer_id,last_active,Recency,Frequency,Monetary,frequency_rank,monetary_rank,recency_segment,frequency_segment,monetary_segment,rfm_segment,rfm_group
0,16738,2022-04-19 09:46:00,305,1,3.0,0.0,0.0,1,1,1,111,Lost
1,16878,2022-11-18 17:39:00,92,2,3.0,0.0,0.0,2,1,1,211,About To Sleep
2,14792,2022-09-11 09:19:00,160,2,5.0,0.0,0.0,2,1,1,211,About To Sleep
3,16454,2022-12-28 11:40:00,52,2,5.0,0.0,0.0,3,1,1,311,Recent Customers
4,14046,2022-05-02 14:48:00,292,1,6.0,0.0,0.0,1,1,1,111,Lost


In [126]:
df = data[['customer_id','Recency','Frequency','Monetary','rfm_group']]
df.head()

,customer_id,Recency,Frequency,Monetary,rfm_group
0,16738,305,1,3.0,Lost
1,16878,92,2,3.0,About To Sleep
2,14792,160,2,5.0,About To Sleep
3,16454,52,2,5.0,Recent Customers
4,14046,292,1,6.0,Lost


In [127]:
#encode label from string to number
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df['rfm_group'])

df['label'] = label_encoder.transform(df['rfm_group'])

<ipython-input-127-48fd7a5aa5be>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = label_encoder.transform(df['rfm_group'])


In [128]:
dic = list(label_encoder.classes_)
dic

['About To Sleep',
 'At Risk',
 'Can’t Lose Them',
 'Champions',
 'Customers Needing Attentions',
 'Hibernating',
 'Lost',
 'Loyal Customers',
 'Potential Loyalist',
 'Promising',
 'Recent Customers']

In [129]:
label_encoder.transform(list(label_encoder.classes_))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [130]:
df.head()

,customer_id,Recency,Frequency,Monetary,rfm_group,label
0,16738,305,1,3.0,Lost,6
1,16878,92,2,3.0,About To Sleep,0
2,14792,160,2,5.0,About To Sleep,0
3,16454,52,2,5.0,Recent Customers,10
4,14046,292,1,6.0,Lost,6


In [131]:
X = df[['Recency','Frequency','Monetary']].values
y = df['label'].values

In [132]:
#scaler data by standard
std = StandardScaler()
std.fit(X)
X = std.transform(X)
X

array([[ 1.66931714e+00, -5.37390780e-02, -6.99011146e-02],
       [-1.79394283e-01, -5.31901207e-02, -6.99011146e-02],
       [ 4.10804670e-01, -5.31901207e-02, -6.98204136e-02],
       ...,
       [-8.91104784e-01,  1.19731425e-01,  6.72221549e+00],
       [-8.12990217e-01,  1.79018812e-01,  8.96796214e+00],
       [-9.17142973e-01,  6.20783447e+01,  6.03452272e+01]])

In [133]:
#create train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.15, random_state = 1)

print('size of dataset train and test: ', y_train.shape, y_test.shape)

size of dataset train and test:  (3304,) (584,)


In [134]:
#train model with GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
xgb =GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
xgb.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [135]:
xgb.score(X_test,y_test)

0.9571917808219178

In [136]:
#predictive function
def predict_customer(Recency, Frequency, Monetary):
    
    label = xgb.predict(std.transform([[Recency,Frequency,Monetary]]))
    label = dic[label[0]]
    return label

In [137]:
print(predict_customer(305,1,3))

Lost


In [138]:
print(predict_customer(50,2,5))

Recent Customers


In [139]:
print(predict_customer(90,150,1700))

Customers Needing Attentions


In [140]:
print(predict_customer(1,5,17000))

Loyal Customers


In [141]:
print(predict_customer(10,395,9000))

Loyal Customers


In [142]:
print(predict_customer(20,339,1736))

Champions
